In [11]:
import sys
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/semi_supervised/')
import coreg
reload(coreg)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ordinal/')
import simple
reload(simple)

import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

from transformers import *
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union

import metrics
reload(metrics)
from sklearn.cross_validation import StratifiedKFold

from sklearn.base import clone

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, ExtraTreesRegressor

import minirank as mr

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from time import time

In [12]:
# Load Data
train_pd  = pd.read_csv('data/train.csv', index_col=0)
test_pd  = pd.read_csv('data/test.csv', index_col=0)

labels = train_pd.Hazard
train_pd.drop('Hazard', axis=1, inplace=True)


columns = train_pd.columns
test_ind = test_pd.index

In [13]:
train = np.array(train_pd)
test = np.array(test_pd)
# label encode the categorical variables
for i in range(train.shape[1]):
    if type(train[1,i]) is str:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[:,i]) + list(test[:,i]))
        train[:,i] = lbl.transform(train[:,i])
        test[:,i] = lbl.transform(test[:,i])

X_train = train.astype(float)
X_test = test.astype(float)
y_train = np.array(labels)

pipe_x = make_pipeline(
    make_union(
#         IdentityTformer(),
#         make_pipeline(AddTformer(1), BoxCoxTformer()),
        AnscombeTformer(),
    ),
    StandardScaler(),
)
pipe_y = make_pipeline(
    LogTformer(),
    StandardScaler(),
)
pipe_x.fit(np.r_[X_train, X_test])


# X_train = pipe_x.transform(X_train)
# X_test = pipe_x.transform(X_test)
# y_train = pipe_y.fit_transform(y_train)

small_n = 500
X_train = X_train[:small_n,:]
y_train = y_train[:small_n]

print y_train.shape
print X_train.shape
print X_test.shape
print len(np.unique(y_train))

(500,)
(500, 32)
(51000, 32)
21


# XGB

In [ ]:
# Parameters for XGB
params = {
    'num_class': 2,
    'silent': 0,    # 0 is verbose
    'nthread': 7,    # max if not set
    
    'bst:eta': 0.01,
    'bst:gamma': 1.0,
    'bst:max_depth': 7,
    'bst:min_child_weight': 5,
    'bst:max_delta_step': 0,
    'bst:subsample': 0.8,    # Row (observation) subsample
    'bst:colsample_bytree': 0.8,    # Col (feature) subsample
#     'bst:scale_pos_weight': 1.0,
    
    'objective':'reg:linear',
    'eval_metric': 'auc',
    
    'seed': 0,
}

dtrain = xgb.DMatrix(train, label=labels)

num_round = 100
scores_xgb = xgb.cv(params, dtrain, num_round, nfold=4,
                    metrics={'auc'}, seed = 0)

In [14]:
clf1 = LinearRegression()
clf2 = ElasticNet()

h = [clone(clf1), clone(clf2)]
clf = coreg.CoReg(h=h, T=5, verbose=True, n_jobs=1)

scores = []
scores_base = []
n_reps = 1
k = 3
for reps in range(n_reps):
    skf = StratifiedKFold(y_train, n_folds=k,
                          shuffle=True,
                          random_state=np.random.randint(0,100))
    for ii, (train, valid) in enumerate(skf):
#         history = LossHistory()   # for keras
        print 'Fold %d' % ii,
        X_train_k = X_train[train]
        X_valid_k = X_train[valid]
        y_train_k = y_train[train]
        y_valid_k = y_train[valid]
        X_all = np.r_[X_train_k, X_valid_k]
        y_all = np.r_[y_train_k, np.nan*np.ones(len(y_valid_k))]
        
        tic = time()
#         clf.fit(X_train_k, y_train_k)
        clf.fit(X_all, y_all)  # for use with coreg
        
        clf1.fit(X_train_k, y_train_k)
        clf2.fit(X_train_k, y_train_k)
        
        toc = time() - tic
        print 'Train time: %2.3f s\t' % toc, 

        valid_preds = clf.predict(X_valid_k)
        valid_base_preds = np.mean([clf1.predict(X_valid_k), clf2.predict(X_valid_k)], axis=0)
        
        score = metrics.gini(y_valid_k, valid_preds)
        score_base = metrics.gini(y_valid_k, valid_base_preds)
        
        print 'Score %1.5f \t %1.5f' % (score, score_base)
        scores.append(score)
        scores_base.append(score_base)
        
        break
        
        
print "done"
print np.array(scores).mean()
print np.array(scores_base).mean()

Fold 0 0Timer unit: 1e-06 s

Total time: 0.002345 s
File: /afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/semi_supervised/coreg.py
Function: calc_d at line 40

Line #      Hits         Time  Per Hit   % Time  Line Contents
    40                                           @profile_each_line
    41                                           def calc_d(clf_type, x_u, h, L, y, nn_tracker, **clf_params):
    42                                               """
    43                                               Calculates the delta, the difference in squared errors between
    44                                                   the regressor with and without the additional pseudolabeled points
    45                                               :param clf_type: type of model for new regressor
    46                                               :param x_u: features of unlabeled observations
    47                                               :param h: the current regressor
    48     

In [10]:
from guppy import hpy
hp = hpy()
h = hp.heap()

h.byrcs[0].byid

Set of 150735 <mixed> objects. Total size = 14921208 bytes.
 Index     Size   %   Cumulative  %   Brief
     0     5400   0.0      5400   0.0 str: '|\x02\x00d)\...\x83\x00\x00S'
     1     5208   0.0     10608   0.1 str: '|\x05\x00r\x...00Sd\x1d\x00S'
     2     4472   0.0     15080   0.1 str: '|\x00\x00j\x...00\x01|5\x00S'
     3     4320   0.0     19400   0.1 str: '|\x00\x00j\x...00Sd\x15\x00S'
     4     4024   0.0     23424   0.2 str: 't\x00\x00j\x...x00|\x0c\x00S'
     5     3696   0.0     27120   0.2 str: '|\x00\x00j\x...\x00Sd\r\x00S'
     6     3416   0.0     30536   0.2 str: '\x88\x00\x00...x00d\x00\x00S'
     7     3288   0.0     33824   0.2 str: '|\x00\x00dy\...0cW|\x05\x00S'
     8     3280   0.0     37104   0.2 str: '|\x0e\x00d\x...00\x01|6\x00S'
     9     3248   0.0     40352   0.3 str: 't\x00\x00}\x...00Sd\x02\x00S'
<150725 more rows. Type e.g. '_.more' to view.>

In [ ]:
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint as sp_randint

param_dist = {
    "n_estimators": sp_randint(10, 800),
    "max_depth": [3, 5, 10, 50, None],
    "max_features": sp_randint(1, X_train.shape[1]),
    "min_samples_split": sp_randint(1, 15),
    "min_samples_leaf": sp_randint(1, 15),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"]}


n_iter_search = 200
random_search = RandomizedSearchCV(
    estimator=ExtraTreesClassifier(), 
    param_distributions=param_dist,
    scoring=metrics.gini_score,
    cv=skf,
    n_iter=n_iter_search,
    n_jobs=-1)

tic = time()
random_search.fit(X_train, y_train)
toc = time() - tic
print toc

In [27]:
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.01
params["min_child_weight"] = 5
params["subsample"] = 0.8
params["colsample_bytree"] = 0.8
params["max_delta_step"] = 10
params["silent"] = 1
params["max_depth"] = 7 #7
params["gamma"] = 0

plst = list(params.items())

#Using 5000 rows for early stopping. 
offset = 5000

num_rounds = 1000
xgtest = xgb.DMatrix(X_test)

#create a train and validation dmatrices 
# xgtrain = xgb.DMatrix(X_train[offset:,:], label=y_train[offset:])
# xgval = xgb.DMatrix(X_train[:offset,:], label=y_train[:offset])
xgtrain = xgb.DMatrix(X_train_k, label=y_train_k)
xgval = xgb.DMatrix(X_valid_k, label=y_valid_k)

#train using early stopping and predict
watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
# preds1 = model.predict(xgtest)


Will train until val error hasn't decreased in 20 rounds.
[0]	train-rmse:5.356750	val-rmse:5.247127
[1]	train-rmse:5.331907	val-rmse:5.222486
[2]	train-rmse:5.306965	val-rmse:5.197869
[3]	train-rmse:5.282235	val-rmse:5.173271
[4]	train-rmse:5.258139	val-rmse:5.149608
[5]	train-rmse:5.233956	val-rmse:5.125635
[6]	train-rmse:5.210391	val-rmse:5.102060
[7]	train-rmse:5.187113	val-rmse:5.079029
[8]	train-rmse:5.164371	val-rmse:5.056501
[9]	train-rmse:5.142098	val-rmse:5.034365
[10]	train-rmse:5.119895	val-rmse:5.012374
[11]	train-rmse:5.098488	val-rmse:4.991148
[12]	train-rmse:5.076819	val-rmse:4.969711
[13]	train-rmse:5.055474	val-rmse:4.948484
[14]	train-rmse:5.034671	val-rmse:4.927866
[15]	train-rmse:5.013700	val-rmse:4.907373
[16]	train-rmse:4.993462	val-rmse:4.887502
[17]	train-rmse:4.973769	val-rmse:4.868074
[18]	train-rmse:4.954302	val-rmse:4.848906
[19]	train-rmse:4.934627	val-rmse:4.829804
[20]	train-rmse:4.915411	val-rmse:4.810984
[21]	train-rmse:4.896283	val-rmse:4.792153
[22]	t

# Generate submission

In [9]:
preds = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds = preds.set_index('Id')
preds.to_csv('xgboost_benchmark.csv')

array([[ 15.,   3.,   2., ...,   4.,   2.,   2.],
       [ 16.,  14.,   5., ...,   4.,   2.,   1.],
       [ 10.,  10.,   5., ...,   4.,   6.,   1.],
       ..., 
       [ 18.,   7.,   5., ...,   4.,   1.,   1.],
       [ 18.,  17.,   5., ...,   2.,   2.,   6.],
       [  5.,  15.,   3., ...,   4.,   5.,   4.]])